In [ ]:
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd

## scrape tickers from yahoo finance

In [ ]:
websites = ["https://finance.yahoo.com/world-indices/",
            #"https://finance.yahoo.com/etfs/"
            ]

# index_info = {}
index_info = pd.DataFrame(columns=['Ticker', 'Name'])

for website in websites:
    r = requests.get(website)
    r.content

    # Use the 'html.parser' to parse the page
    soup = BeautifulSoup(r.content, 'html.parser')

    # Find the table with the columns: ticker and Name
    table = soup.find('table')
    if table:
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            print(f"Cols: {cols[0].text}")
            cols = [ele.text for ele in cols]
            # index_info[cols[0]] = cols[1]
            index_info.loc[len(index_info)] = [cols[0], cols[1:]]

# print(json.dumps(index_info, indent=4))
print(index_info)

## get stock tickers from nasdaq

In [ ]:
import ftplib

In [ ]:
filenames = ["nasdaqlisted.txt", "otherlisted.txt"]

ftp_server = ftplib.FTP("ftp.nasdaqtrader.com")
ftp_server.login()
ftp_server.encoding = 'utf-8'

ftp_server.cwd("SymbolDirectory")
ftp_server.dir()

for filename in filenames:
    with open(filename, 'wb') as file:
        ftp_server.retrbinary('RETR ' + filename, file.write)

ftp_server.quit()

In [ ]:
otherlisted = pd.read_csv("otherlisted.txt", sep='|')
nasdaqlisted = pd.read_csv("nasdaqlisted.txt", sep='|')

print(otherlisted.head())

In [ ]:
otherlisted[otherlisted['ETF'] == 'Y']

In [ ]:
# find a stock in the df's
stock_name = 'msci'

def find_stock(stock_name, stock_data):
    # drop na values
    stock_data.dropna(inplace=True)
    
    print(stock_data[stock_data['Security Name'].str.contains(stock_name, case=False)])
    print("\n")

find_stock(stock_name, otherlisted)
find_stock(stock_name, nasdaqlisted)

In [ ]:
otherlisted_clean = otherlisted.dropna(subset=['Security Name'])
otherlisted_clean[otherlisted_clean['Security Name'].str.contains('World')]

## SymbolProvider

In [ ]:
from mc_simulation.data_provider import SymbolReader

In [ ]:
provider = SymbolReader(["../jupyter_notebooks/otherlisted.txt", "../jupyter_notebooks/nasdaqlisted.txt"])
symbols = provider.read_symbols()
symbols = provider.process_symbols(symbols)
print(symbols)